In [7]:
import os
import gc

import copy
import pandas as pd
import numpy as np
import math
import random
import datetime
import itertools

from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, QuantileTransformer
from sklearn import decomposition, cluster

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.weight_norm import WeightNorm

from scipy import stats

import multiprocessing
import joblib
from joblib import Parallel, delayed

import pickle

# Parameters

In [ ]:
is_submission = False
is_kaggle = True
is_train_without_holdout = True

if is_kaggle:
    import sys
    sys.path.append('../input/iterative-stratification/iterative-stratification-master') 
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    path_data = '../input/lish-moa/'
    path_output = ''
else:
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    path_data = 'data/'
    path_output = 'data/'

if is_submission:
    validation_ratio = 0.0
    test_features = pd.read_csv(path_data+'test_features.csv')
    run_script = test_features.shape[0] > 3982
    if run_script==False:
        !cp ../input/lish-moa/sample_submission.csv .
        !mv ./sample_submission.csv ./submission.csv
else:
    validation_ratio = 0.2
    run_script = True
    
if is_train_without_holdout:
    validation_ratio = 0.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

calculate_new_prepared_data = False
dump_prepared_data = False
prepared_data_version = 'p21'

train_model = True
if train_model:
    path_model = ''
else:
    if is_kaggle:
        path_model = '../input/'+'modelp21lb/'
    else:
        path_model = ''


SEED = 42
SHIFT = 2222

n_seeds = 5

cpu = multiprocessing.cpu_count()

folds = 5
fold_type = None # 'multi' but without drugs separation

oof_type = 'multi' #can be 'kfold' or 'multi'
oof_threshold = 18 

add_kernels = False
use_log_for_kernel_diff = True
inverse_kde = False
ratio_inverse_kde = True
use_diff_kde = False #When set to true, use_log_for_kernel_diff, inverse_kde, ratio_inverse_kde become irrelevant
exclude_c_from_kde = False
exclude_g_from_kde = True

g_removal_count = 75 #the higher the more columns to drop
add_c_stats = False


perform_pca = False
pca_for_c = False
pca_for_kde = False

use_train_test_for_norm = True
normalization_type = 'standard' #standard or quantile

control_share_in_train = 0.0
add_control_from_test = False

augment_data = False
additional = 0 #won't calculate if zero
granularity = 100
max_dev = 0.1
normal_std_dev = 0.1

label_smoothing = 0.00025
    
model_nonscored = False

oof_loss_limit = 0.0

nn_architecture = 'v1.2'

In [ ]:
exclusivity_tuples = [
 ['potassium_channel_activator',
  'potassium_channel_antagonist',
  'potassium_channel_agonist',
  'potassium_channel_blocker'],
 ['atp-sensitive_potassium_channel_antagonist',
  'atp-sensitive_potassium_channel_agonist',
  'atp-sensitive_potassium_channel_inhibitor'],
 ['gaba_receptor_agonist',
  'gaba_receptor_modulator'],
 ['glutamate_receptor_agonist',
  'glutamate_receptor_antagonist',
  'glutamate_receptor_modulator'],
 ['nitric_oxide_donor',
  'nitric_oxide_scavenger',
  'nitric_oxide_stimulant'],
 ['prostanoid_receptor_antagonist',
  'prostanoid_receptor_agonist',
 ' prostanoid_receptor_inhibitor'],
 ['sodium_channel_inhibitor',
  'sodium_channel_activator',
  'sodium_channel_blocker'],
 ['acetylcholine_receptor_agonist',
  'acetylcholine_receptor_antagonist'],
 ['adenosine_receptor_agonist',
  'adenosine_receptor_antagonist'],
 ['adenylyl_cyclase_activator',
  'adenylyl_cyclase_inhibitor'],
 ['adrenergic_receptor_agonist',
  'adrenergic_receptor_antagonist'],
 ['aldehyde_dehydrogenase_inhibitor',
  'aldehyde_dehydrogenase_activator'],
 ['ampk_activator',
  'ampk_inhibitor'],
 ['androgen_receptor_agonist',
  'androgen_receptor_antagonist'],
 ['angiotensin_receptor_antagonist',
  'angiotensin_receptor_agonist'],
 ['apoptosis_stimulant', 
  'apoptosis_inhibitor'],
 ['aryl_hydrocarbon_receptor_agonist', 
  'aryl_hydrocarbon_receptor_antagonist'],
 ['atp_channel_activator', 
  'atp_channel_blocker'],
 ['benzodiazepine_receptor_agonist', 
  'benzodiazepine_receptor_antagonist'],
 ['calcium_channel_blocker',
  'calcium_channel_activator'],
 ['cannabinoid_receptor_agonist', 
  'cannabinoid_receptor_antagonist'],
 ['car_agonist', 
  'car_antagonist'],
 ['caspase_activator', 
  'caspase_inhibitor'],
 ['cc_chemokine_receptor_antagonist',
  'cc_chemokine_receptor_agonist'],
 ['cftr_channel_agonist', 
  'cftr_channel_antagonist'],
 ['chloride_channel_blocker',
  'chloride_channel_activator'],
 ['cholinergic_receptor_antagonist',
  'cholinergic_receptor_agonist'],
 ['complement_antagonist', 
  'complement_inhibitor'],
 ['corticosteroid_agonist',
  'corticosteroid_antagonist'],
 ['dopamine_receptor_agonist',
  'dopamine_receptor_antagonist'],
 ['estrogen_receptor_agonist',
  'estrogen_receptor_antagonist'],
 ['fatty_acid_receptor_agonist',
  'fatty_acid_receptor_antagonist'],
 ['fxr_agonist', 
  'fxr_antagonist'],
 ['g_protein-coupled_receptor_agonist',
  'g_protein-coupled_receptor_antagonist'],
 ['glucocorticoid_receptor_agonist',
  'glucocorticoid_receptor_antagonist'],
 ['glucokinase_activator',
  'glucokinase_inhibitor'],
 ['gonadotropin_receptor_agonist',
  'gonadotropin_receptor_antagonist'],
 ['guanylate_cyclase_activator',
  'guanylate_cyclase_stimulant'],
 ['histamine_receptor_agonist',
  'histamine_receptor_antagonist'],
 ['hsp_inhibitor',
  'hsp_inducer'],
 ['icam1_antagonist',
  'icam1_inhibitor'],
 ['membrane_permeability_enhancer',
  'membrane_permeability_inhibitor'],
 ['mineralocorticoid_receptor_antagonist',
  'mineralocorticoid_receptor_agonist'],
 ['neurotensin_receptor_agonist',
  'neurotensin_receptor_antagonist'],
 ['nfkb_inhibitor', 
  'nfkb_activator'],
 ['opioid_receptor_agonist',
  'opioid_receptor_antagonist'],
 ['oxytocin_receptor_agonist',
  'oxytocin_receptor_antagonist'],
 ['p53_activator',
  'p53_inhibitor'],
 ['phospholipase_inhibitor',
  'phospholipase_activator'],
 ['pka_activator',
  'pka_inhibitor'],
 ['ppar_receptor_agonist',
  'ppar_receptor_antagonist'],
 ['progesterone_receptor_agonist',
  'progesterone_receptor_antagonist'],
 ['protein_kinase_inhibitor', 
  'protein_kinase_activator'],
 ['protein_synthesis_inhibitor',
  'protein_synthesis_stimulant'],
 ['retinoid_receptor_agonist', 
  'retinoid_receptor_antagonist'],
 ['serotonin_receptor_agonist', 
  'serotonin_receptor_antagonist'],
 ['sigma_receptor_agonist', 
  'sigma_receptor_antagonist'],
 ['sirt_activator',
  'sirt_inhibitor'],
 ['smoothened_receptor_antagonist',
  'smoothened_receptor_agonist'],
 ['src_inhibitor',
  'src_activator'],
 ['thyroid_hormone_inhibitor',
  'thyroid_hormone_stimulant'],
 ['tlr_agonist',
  'tlr_antagonist'],
 ['trace_amine_associated_receptor_agonist',
  'trace_amine_associated_receptor_antagonist'],
 ['transient_receptor_potential_channel_antagonist',
  'transient_receptor_potential_channel_agonist'],
 ['trpv_agonist',
  'trpv_antagonist'],
 ['urotensin_receptor_agonist',
  'urotensin_receptor_antagonist'],
 ['vasopressin_receptor_agonist',
  'vasopressin_receptor_antagonist'],
 ['wnt_inhibitor',
  'wnt_agonist']
]

# Helper functions

In [1]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True
    
seed_everything(seed=SEED)

NameError: name 'utils' is not defined

In [ ]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size


def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

In [ ]:
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Model functions

In [ ]:
class prepareData:
    def __init__(self,path,validation_ratio=0,folds=5,
                 g_removal_count=0, add_c_stats=False, normalization_type='standard',
                 add_kernels=False, use_log_for_kernel_diff=False, inverse_kde=False, ratio_inverse_kde=False, use_diff_kde=False,exclude_c_from_kde=False,exclude_g_from_kde=False,
                 perform_pca=False, pca_variance_threshold=0.95, pca_for_kde=False, pca_for_c=True,
                 use_train_test_for_norm=True,cpu=None,
                 granularity=100,max_dev=0.1,normal_std_dev=0.1,additional=10):
        self.path = path
        self.folds = 5
        self.use_log_for_kernel_diff = use_log_for_kernel_diff
        self.normalization_type = normalization_type
        self.add_kernels = add_kernels
        self.add_c_stats = add_c_stats
        self.perform_pca = perform_pca
        self.pca_variance_threshold = pca_variance_threshold
        self.use_log_for_kernel_diff = use_log_for_kernel_diff
        self.inverse_kde = inverse_kde
        self.use_diff_kde = use_diff_kde
        self.exclude_c_from_kde = exclude_c_from_kde
        self.exclude_g_from_kde = exclude_g_from_kde
        
        
        if cpu is None:
            cpu = multiprocessing.cpu_count()
            self.cpu = cpu
        else:
            cpu = min(cpu,multiprocessing.cpu_count())
            self.cpu = cpu
            
        print('import data')
        self._import_data(self.path)
        self._num_features = list(set(self.X_train.columns) - set(['sig_id','cp_type','cp_dose','cp_time']))
        
        print('transform cat features')
        self.X_train = self._transform_cat_features(self.X_train)
        self.X_test = self._transform_cat_features(self.X_test)
        
        print('kde kernels calculations')
        self.kde_kernels = self._calculate_kde_kernels(self.X_train,self.X_test,ratio_inverse_kde)
        
        print('remove g columns with low variation')
        self.g_to_drop = self._calculate_g_cols_to_drop(self._num_features,self.kde_kernels,g_removal_count)
        self.X_train.drop(self.g_to_drop,inplace=True,axis=1)
        self.X_test.drop(self.g_to_drop,inplace=True,axis=1)
        self._num_features = list(set(self.X_train.columns) - set(['sig_id','cp_type','cp_dose','cp_time']))
        
        if add_kernels:
            print('kde features')
            self.X_train = self._process_kde_parallelized(self.X_train,self.kde_kernels,use_log_for_kernel_diff,inverse_kde,use_diff_kde,cpu,exclude_c_from_kde,exclude_g_from_kde)
            self.X_test = self._process_kde_parallelized(self.X_test,self.kde_kernels,use_log_for_kernel_diff,inverse_kde,use_diff_kde,cpu,exclude_c_from_kde,exclude_g_from_kde)
            
        if add_c_stats:
            print('add survavilability stats (c)')
            self.X_train = self._add_c_stats(self.X_train)
            self.X_test = self._add_c_stats(self.X_test)
            
        if perform_pca:
            print('perform pca')
            self._fit_pca([self.X_train,self.X_test],pca_for_kde,pca_for_c)
            self.X_train = self._transform_pca(self.X_train,pca_variance_threshold)
            self.X_test = self._transform_pca(self.X_test,pca_variance_threshold)
        
        print('normalize features')
        if use_train_test_for_norm:
            _ = self._normalize_features(pd.concat([self.X_train,self.X_test],axis=0))
            self.X_train = self._normalize_features(self.X_train,is_test=True)
            self.X_test = self._normalize_features(self.X_test,is_test=True)
        else:
            self.X_train = self._normalize_features(self.X_train)
            self.X_test = self._normalize_features(self.X_test,is_test=True)
            
        
        if additional>0:
            print('data augmentation')
            X_list = [self.X_train,self.X_test]

            prob_dist = self._calculate_prob_dist_for_data_augmentation(X_list,granularity)
            self.var_list = self._generate_data_augmentation(self.X_train,prob_dist,granularity,max_dev=max_dev,normal_std_dev=normal_std_dev,additional=additional)
        
    
    def prepare_test_data(self,path):
        X_test = pd.read_csv(self.path+'test_features.csv')
        X_test = self._transform_cat_features(X_test)
        X_test.drop(self.g_to_drop,inplace=True,axis=1)
        if self.add_kernels:
            X_test = self._process_kde_parallelized(X_test,self.kde_kernels,self.use_log_for_kernel_diff,self.inverse_kde,self.use_diff_kde,self.cpu,self.exclude_c_from_kde,self.exclude_g_from_kde)
        if self.add_c_stats:
            X_test = self._add_c_stats(X_test)
        if self.perform_pca:
            X_test = self._transform_pca(X_test,self.pca_variance_threshold)
        X_test = self._normalize_features(X_test,is_test=True)
        
        self.X_test = X_test
        
        
        
    def _index_to_boolean(self,idx,size):
        mask_array = np.zeros(size)
        mask_array = mask_array==1
        mask_array[idx] = True
        return mask_array
    def _boolean_to_index(self,boolean_array):
        idx = np.arange(0,boolean_array.shape[0])
        idx = idx[boolean_array].copy()
        return idx
        
    def _import_data(self,path):
        self.X_train = pd.read_csv(path+'train_features.csv')
        self.X_test = pd.read_csv(path+'test_features.csv')
        self.y_train = pd.read_csv(path+'train_targets_scored.csv')
        self.X_train_additional = pd.read_csv(path+'train_targets_nonscored.csv')
        self.X_train_drugs = pd.read_csv(path+'train_drug.csv')
        self.sample_submission = pd.read_csv(path+'sample_submission.csv')
        
        
    def _transform_cat_features(self,X):
        X['cp_type'] = X['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})
        X['cp_dose'] = X['cp_dose'].map({'D1':0,'D2':1})
        X['cp_time'] = X['cp_time'].map({24:0,48:0.5,72:1})
        return X
    
    def _normalize_features(self,X,is_test=False):
        cols_to_normalize = list(set(self.X_train.columns) - set(['sig_id','cp_type','cp_dose','cp_time']))
        if is_test==False:
            self.normalizer_dict = {}
        for col in cols_to_normalize:
            if is_test:
                scaler = self.normalizer_dict[col]
                a = X[col].values.reshape(-1,1)
                a = (scaler.transform(a)).flatten()
                
                if (self.normalization_type == 'quantile') and (not '_kde_diff' in col):
                    a = a/10.0 + 0.5
                X[col] = a
            else:
                if (self.normalization_type == 'quantile') and (not '_kde_diff' in col):
                    scaler = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
                else:
                    scaler = MinMaxScaler()
                
                a = X[col].values
                a = scaler.fit_transform(a.reshape(-1, 1))
                self.normalizer_dict[col] = scaler
                
                if (self.normalization_type == 'quantile') and (not '_kde_diff' in col):
                    #Quantilization transforms data on a [-5:+5] range here
                    a = a/10.0 + 0.5
                    
                X[col] = a
                
        return X
    
    def _calculate_kde_kernels(self,X1,X2,ratio_inverse_kde):
        X = pd.concat([X1,X2])
        X_control = X[X['cp_type']==1]
        X_treatment = X[X['cp_type']==0]
        kernels = {}
        cols = self._num_features
        for col in cols:
            #Calculate kernels
            x_control = X_control[col].values
            x_treatment = X_treatment[col].values
            kde_control_kernel = stats.gaussian_kde(x_control)
            kde_treatment_kernel = stats.gaussian_kde(x_treatment)
            kernels[col+'_control'] = kde_control_kernel
            kernels[col+'_treatment'] = kde_treatment_kernel
            
            #Calculate max ratio so that when calculating kde features based on the ratio of treatement/control, we have a threshold for values
            x_control_mean = x_control.mean()
            x_control_std = x_control.std()
            x_treatment_mean = x_treatment.mean()
            #As b is not usually normal we use only a std to create range
            kde_range = [min(x_control_mean - 2*x_control_std, x_treatment_mean - 2*x_control_std),max(x_control_mean + 2*x_control_std, x_treatment_mean + 2*x_control_std)]
            kde_sample = np.arange(kde_range[0],kde_range[1],(kde_range[1]-kde_range[0])/100)
            
            x_control_kde_sample = kde_control_kernel.pdf(kde_sample)
            x_treatment_kde_sample = kde_treatment_kernel.pdf(kde_sample)
            if ratio_inverse_kde:
                max_ratio = (x_control_kde_sample/x_treatment_kde_sample).max()
            else:
                max_ratio = (x_treatment_kde_sample/x_control_kde_sample).max()
            kernels[col+'_ratio'] = max_ratio
            
        return kernels
    
    def _build_batch(self,X,kernels,use_log_for_kernel_diff,inverse_kde,use_diff_kde,cpu_count,exclude_c_from_kde,exclude_g_from_kde):
        batch_list = []
        cols = self._num_features
        if exclude_c_from_kde:
            cols = [col for col in cols if not 'c-' in col]
        if exclude_g_from_kde:
            cols = [col for col in cols if not 'g-' in col]
        col_size = len(cols)            

        if col_size>=cpu_count:
            batch_size = int(col_size/cpu_count)
        else:
            batch_size = 1
            cpu_count = col_size
        for i in range(cpu_count):
            if i == cpu_count-1:
                batch_list.append((cols[i*batch_size:],X,kernels,use_log_for_kernel_diff,inverse_kde,use_diff_kde))
            else:
                batch_list.append((cols[i*batch_size:(i+1)*batch_size],X,kernels,use_log_for_kernel_diff,inverse_kde,use_diff_kde))
        return batch_list

    def _process_individual_batch(self,batch):
        ratio_multiplier = 10
        cols = batch[0]
        X = batch[1]
        kernels = batch[2]
        use_log_for_kernel_diff = batch[3]
        inverse_kde = batch[4]
        use_diff_kde = batch[5]
        series_list = []
        for col in cols:
            kde_control_kernel = kernels[col+'_control']
            kde_treatment_kernel = kernels[col+'_treatment']
            
            if use_diff_kde:
                a_kde = kde_control_kernel.pdf(X[col].values)
                b_kde = kde_treatment_kernel.pdf(X[col].values)
                a = (b_kde-a_kde)/np.max((a_kde,b_kde),axis=0)
                a = a.clip(-1,1)
                a = np.nan_to_num(a,nan=0.0)
            else:
                if inverse_kde:
                    a = kde_control_kernel.pdf(X[col].values)/kde_treatment_kernel.pdf(X[col].values)
                else:
                    a = kde_treatment_kernel.pdf(X[col].values)/kde_control_kernel.pdf(X[col].values)
                a = np.nan_to_num(a,nan=ratio_multiplier*kernels[col+'_ratio'])
                a = a.clip(0,ratio_multiplier*kernels[col+'_ratio'])
                if use_log_for_kernel_diff:
                    a = np.log1p(a)
                    
            a = pd.Series(a,name=col+'_kde_diff',dtype='float32')
            series_list.append(a)
        return series_list

    def _run_batch(self,batch):
        return self._process_individual_batch(batch)

    def _process_batch_list(self,batch_list,cpu):
        return joblib.Parallel(n_jobs=cpu)(joblib.delayed(self._run_batch)(batch) for batch in batch_list)

    def _process_kde_parallelized(self,X,kernels,use_log_for_kernel_diff,inverse_kde,use_diff_kde,cpu,exclude_c_from_kde,exclude_g_from_kde):
        batch_list = self._build_batch(X,kernels,use_log_for_kernel_diff,inverse_kde,use_diff_kde,cpu,exclude_c_from_kde,exclude_g_from_kde)
        results = self._process_batch_list(batch_list,cpu)
        for series_list in results:
            for s in series_list:
                X[s.name] = s.values
        return X
    
    def _calculate_prob_dist_for_data_augmentation(self,X_list,granularity):
        X = pd.concat(X_list)
        X_treatment = X[X['cp_type']==0]
        prob_dist = []
        dist = np.arange(0,1,1/granularity)
        for col in X.columns[4:]:
            x = X_treatment[col].values
            kernel = stats.gaussian_kde(x)
            prob = kernel.pdf(dist)
            prob_dist.append(prob)
        prob_dist = np.array(prob_dist)
        return prob_dist
    
    def _generate_data_augmentation(self,X,prob_dist,granularity,max_dev=0.1,normal_std_dev=0.1,additional=10):
        rng = np.random.default_rng(seed=42)
        x = X.values[:,4:].copy().astype(np.float16)

        max_dev_steps = int(max_dev*granularity)

        #Calculate normal matrix
        normal_p = np.arange(-max_dev*granularity,max_dev*granularity+1,1)
        normal_p = normal_p/granularity
        normal_p = 1/(normal_std_dev)*np.exp(-(normal_p*normal_p)/normal_std_dev**2)
        normal_p = normal_p.astype(np.float16)
        normal_p = np.repeat(normal_p[np.newaxis,:], x.shape[1], axis=0)
        normal_p = np.repeat(normal_p[np.newaxis,:,:], x.shape[0], axis=0)

        #Transform x so that it rounds to the desired granularity
        x_rounded = (np.round(x*granularity)).astype(int)


        #For each and every value a in x, we want to calculate a vector of probability of size 2n+1 such as
        #The probability value at index 0 is the probability that we remove max_dev to a
        i_steps = np.arange(-max_dev_steps,max_dev_steps+1,1) #initialization vector for the steps
        i_initial = np.tile(np.array([[i_steps]]),(x.shape[0],x.shape[1],1))
        x_rounded_repeated = np.repeat(x_rounded[:, :, np.newaxis], i_steps.shape[0], axis=2)
        idx = i_initial + x_rounded_repeated
        idx = idx.copy()
        idx = np.clip(idx,0,granularity-1) #For each 
        
        del i_initial, x_rounded_repeated
        gc.collect()

        #prob_candidates = prob_dist[0,0,idx].copy()
        prob_candidates = np.zeros(idx.shape)
        for j in range(idx.shape[1]):
            prob_candidates[:,j,:] = np.take(prob_dist[j,:],idx[:,j,:])
        
        del idx
        gc.collect()
        
        prob_candidates = prob_candidates*normal_p
        prob_candidates = prob_candidates.copy()

        del normal_p
        gc.collect()

        prob_candidates = prob_candidates/prob_candidates.sum(axis=2)[:,:,np.newaxis]


        var = np.zeros([x.shape[0],x.shape[1],additional])
        i_steps_norm = i_steps/max_dev_steps*max_dev
        print('calculating probas')
        for k in range(x.shape[1]):
            for i in range(x.shape[0]):
                var[i,k,:] = np.random.choice(i_steps_norm,size=additional,p=prob_candidates[i,k,:])

        var_list = []
        for i in range(additional):
            var_list.append(var[:,:,i].copy())
        return var_list
    
    
    def _calculate_g_cols_to_drop(self,cols,kde_kernels,g_removal_count):
        g_to_drop = []
        if g_removal_count==0:
            return g_to_drop
        g_cols = []
        diff_list = []
        for col in cols:
            if 'g-'==col[:2]:
                g_cols.append(col)
                
                kde_control_kernel = kde_kernels[col+'_control']
                kde_treatment_kernel = kde_kernels[col+'_treatment']
                
                rg = np.arange(-10,10,0.1)
                
                vehicle_kde_sample = kde_treatment_kernel.pdf(rg)
                control_kde_sample = kde_control_kernel.pdf(rg)
                
                diff = (np.abs(vehicle_kde_sample-control_kde_sample)).mean()
                
                diff_list.append(diff)

        diff_list_ordered = np.sort(np.array(diff_list))
        thresh = diff_list_ordered[g_removal_count-1]
        
        for col, diff in zip(g_cols,diff_list):
            if diff<=thresh:
                g_to_drop.append(col)
                
        return g_to_drop
    
    def _add_c_stats(self,X):
        all_cols = X.columns
        c_cols = [x for x in all_cols if ('c-' in x) & (not '_kde_diff' in x)]
        X_values = X[c_cols].values
        #Add stats
        X['c_stats_sum'] = X_values.sum(axis=1)
        X['c_stats_mean'] = X_values.mean(axis=1)
        X['c_stats_median'] = np.median(X_values,axis=1)
        X['c_stats_std'] = np.std(X_values,axis=1)
        X['c_stats_kurtosis'] = stats.kurtosis(X_values,axis=1)
        X['c_stats_skew'] = stats.skew(X_values,axis=1)
        
        return X
    
    def _fit_pca(self,X_list,pca_for_kde,pca_for_c):
        X = pd.concat(X_list,axis=0)
        all_cols = X.columns
        pca_cols = []
        pca_names = ['g_pca']
        if pca_for_c:
            pca_names.append('c_pca')
        pca_cols.append([x for x in all_cols if ('g-' in x) & (not '_kde_diff' in x)])
        if pca_for_c:
            pca_cols.append([x for x in all_cols if ('c-' in x) & (not '_kde_diff' in x) & (not '_stats' in x)])
        if pca_for_kde:
            pca_cols.append([x for x in all_cols if ('g-' in x) & ('_kde_diff' in x)])
            if pca_for_c:
                pca_cols.append([x for x in all_cols if ('c-' in x) & ('_kde_diff' in x) & (not '_stats' in x)])
            pca_names.append('g_kde_pca')
            if pca_for_c:
                pca_names.append('c_kde_pca')

        self.pca_cols_dict = {}
        self.pca_dict = {}
        for name,cols in zip(pca_names,pca_cols):
            if len(cols)>0:
                X_pca = X[cols]
                pca = decomposition.PCA(n_components=X_pca.shape[1],
                                          whiten=True,
                                          svd_solver='full',
                                          random_state=42
                                         )
                pca.fit(X_pca)
                self.pca_cols_dict[name] = cols
                self.pca_dict[name] = pca    
                
                
    def _calculate_pca_components_to_keep(self,explained_variance_ratio_,pca_variance_threshold):
        explained_variance_ratio_cum = explained_variance_ratio_.cumsum()
        return np.argmax(explained_variance_ratio_cum>=pca_variance_threshold) + 1

    def _transform_pca(self,X,pca_variance_threshold):
        pca_names = list(self.pca_cols_dict.keys())
        for name in pca_names:
            #Recover cols and fit pca
            cols = self.pca_cols_dict[name]
            pca = self.pca_dict[name]

            #Transform to current data
            X_pca = pca.transform(X[cols])

            #Keep only necessary data + transform into pd
            variance_limit = self._calculate_pca_components_to_keep(pca.explained_variance_ratio_,pca_variance_threshold)
            X_pca = X_pca[:,:variance_limit]
            new_cols = [name+'_'+str(i) for i in range(variance_limit)]
            X_pca = pd.DataFrame(X_pca,columns=new_cols)

            #Adjust X
            X.drop(cols,axis=1,inplace=True)
            X = pd.concat([X,X_pca],axis=1)

        return X
    
    def _duplicate_data_for_imbalanced_labels(self,X,y,folds):
        cols_with_not_enough_data = np.where(y.iloc[:,1:].sum().values<folds)[0]
        for col_index in cols_with_not_enough_data:
            rows = np.where(y.iloc[:,col_index+1].values==1)[0]
            n_rows = rows.shape[0]
            if n_rows > 0:
                n_duplicates = folds//n_rows + 1
                X_duplicate_pd = X.iloc[rows,:].copy()
                y_duplicate_pd = y.iloc[rows,:].copy()
                X = pd.concat([X] + [X_duplicate_pd]*n_duplicates)
                y = pd.concat([y] + [y_duplicate_pd]*n_duplicates)
            
        return X,y
    
    
    def _add_nonscored_targets(self,X):
        X = pd.merge(X,self.X_train_additional,on='sig_id')
        y = X[self.X_train_additional.columns].copy()
        return X,y

    
    def _pong_number(self,a,size):
        divisor = a//size
        remaining = a%size
        is_even = divisor % 2 == 0
        if is_even:
            return remaining
        else:
            return size-remaining
    
    def _select_split_drugs(self,X,ratio):
        treatment_drugs = X['drug_id'].values
        drug_ids, drug_counts = np.unique(treatment_drugs, return_counts=True)
        drug_id_counts = np.array([drug_ids,drug_counts]).transpose()
        drug_id_counts = drug_id_counts[drug_id_counts[:,1].argsort()]
        n_drugs = drug_id_counts.shape[0]
        
        #Randomly chose drugs that will appear only in holdout
        holdout_drugs = []
        array_size = X.shape[0]
        limit = int(array_size*ratio)
        count_drugs = 0
        random_range = int(1/ratio)+1
        counter = 0
        
        
        while count_drugs<limit:
            choice = random_range*counter + np.random.randint(0,random_range)
            choice = self._pong_number(choice,n_drugs-1)
            holdout_drugs.append(drug_id_counts[choice,0])
            count_drugs += drug_id_counts[choice,1]
            #holdout_drugs.append(np.take(drug_id_counts,choice,mode='wrap',axis=0)[0])
            #count_drugs += np.take(drug_id_counts,choice,mode='wrap',axis=0)[1]
            counter += 1
            
        return holdout_drugs
            
        
    def split_train_holdout(self,validation_ratio=validation_ratio,seed=42,threshold=1000):
        if validation_ratio==0:
            X_train = self.X_train.copy()
            X_train = pd.merge(X_train,self.y_train,on='sig_id')
            X_train = pd.merge(X_train,self.X_train_drugs,on='sig_id')
            X_drugs = X_train['drug_id'].values
            X_sig_id = X_train['sig_id'].values
            y_train = X_train[self.y_train.columns.tolist()].iloc[:,1:].copy()
            X_train = X_train[self.X_train.columns.tolist()].iloc[:,1:].copy()
            mask_train = np.isin(self.X_train['sig_id'].values,X_sig_id)
            return  X_train,y_train,X_drugs,X_sig_id,None,None,mask_train
        else:
            seed_everything(seed)
            X = self.X_train.copy()
            X = pd.merge(X,self.y_train,on='sig_id')
            X = pd.merge(X,self.X_train_drugs,on='sig_id')

            #Split control in 2
            mask_control = X.iloc[:,1].values==1
            idx_control = self._boolean_to_index(mask_control)

            idx_control_train, idx_control_holdout = train_test_split(idx_control,test_size=validation_ratio,random_state=seed)
            X_control_train = X.iloc[idx_control_train]
            X_control_holdout = X.iloc[idx_control_holdout]

            #Identify drugs above threshold
            drugs_count = X['drug_id'].value_counts()
            drugs_above_thresh = drugs_count.loc[drugs_count>threshold].index.values.tolist()
            drugs_above_thresh = list(set(drugs_above_thresh) - set(X[X['cp_type']==1]['drug_id'].values.tolist()))
            if len(drugs_above_thresh)>0:
                idx_drugs_spread = X[X['drug_id'].isin(drugs_above_thresh)].index.values.tolist()
                mask_drugs_spread = self._index_to_boolean(idx_drugs_spread,X.shape[0])

                idx_drugs_spread_train, idx_drugs_spread_holdout = train_test_split(idx_drugs_spread,test_size=validation_ratio,random_state=seed)
                X_drugs_spread_train = X.iloc[idx_drugs_spread_train]
                X_drugs_spread_holdout = X.iloc[idx_drugs_spread_holdout]

            #Split treatment in 2 by respecting that drugs only appear on 1 group
            #Create list of drugs and how many records they have
            if len(drugs_above_thresh)>0:
                mask_treatment = (X.iloc[:,1].values==0) & (mask_drugs_spread==False)
            else:
                mask_treatment = (X.iloc[:,1].values==0)
            holdout_drugs = self._select_split_drugs(X[mask_treatment].copy(),validation_ratio)
            
            if len(drugs_above_thresh)==0:
                X_train = pd.concat([X[~(X['drug_id'].isin(holdout_drugs)) & (X['cp_type']==0)],X_control_train],axis=0)
                X_holdout = pd.concat([X[(X['drug_id'].isin(holdout_drugs)) & (X['cp_type']==0)],X_control_holdout],axis=0)
            else:
                X_train = pd.concat([X[~(X['drug_id'].isin(holdout_drugs)) & (X['cp_type']==0) & ~(X['drug_id'].isin(drugs_above_thresh))],X_control_train,X_drugs_spread_train],axis=0)
                X_holdout = pd.concat([X[(X['drug_id'].isin(holdout_drugs)) & (X['cp_type']==0)],X_control_holdout,X_drugs_spread_holdout],axis=0)
            
            #reorder
            X_train = pd.merge(self.X_train['sig_id'],X_train,on=['sig_id'])
            
            y_train = X_train[self.y_train.columns.tolist()].iloc[:,1:].copy()
            y_holdout = X_holdout[self.y_train.columns.tolist()].iloc[:,1:].copy()
            
            X_drugs = X_train['drug_id'].values
            X_sig_id = X_train['sig_id'].values
            X_train = X_train[self.X_train.columns.tolist()].iloc[:,1:].copy()
            X_holdout = X_holdout[self.X_train.columns.tolist()].iloc[:,1:].copy()
            mask_train = np.isin(self.X_train['sig_id'].values,X_sig_id)
            
            return X_train,y_train,X_drugs,X_sig_id,X_holdout,y_holdout,mask_train
        
    def create_cv(self,X,y,drugs,sig_ids,threshold=1000,folds=folds,seed=42):
        seed_everything(seed)

        y_cols = y.columns.tolist()
        X = X.copy()
        y = y.copy()

        X = pd.concat([X,y],axis=1)
        X['drug_id'] = drugs
        X['sig_id'] = sig_ids

        #Locate drugs
        drugs_count = X['drug_id'].value_counts()
        drugs_below_thresh = drugs_count.loc[drugs_count<=threshold].index.sort_values()
        drugs_above_thresh = drugs_count.loc[drugs_count>threshold].index.sort_values()

        dct_below_thresh = {}; dct_above_thresh = {}
        #Stratify below threshold
        skf= MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        tmp = X.groupby('drug_id')[y_cols].mean().loc[drugs_below_thresh]
        for f,(idxT,idxV) in enumerate( skf.split(tmp,tmp[y_cols])):
            dd = {k:f for k in tmp.index[idxV].values}
            dct_below_thresh.update(dd)

        #stratify above threshold
        skf= MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        tmp = X.loc[X['drug_id'].isin(drugs_above_thresh)].reset_index(drop=True)
        for f,(idxT,idxV) in enumerate( skf.split(tmp,tmp[y_cols])):
            dd = {k:f for k in tmp.sig_id[idxV].values}
            dct_above_thresh.update(dd)

        # ASSIGN FOLDS
        X['fold'] = X['drug_id'].map(dct_below_thresh)
        X.loc[X['fold'].isna(),'fold'] = X.loc[X['fold'].isna(),'sig_id'].map(dct_above_thresh)
        X['fold'] = X['fold'].astype('int8')
        
        oof_assignment = X['fold'].values
        
        oof_idx = []
        for x in np.arange(folds):
            train = np.where(oof_assignment!=x)[0]
            val = np.where(oof_assignment==x)[0]
            oof_idx.append((train,val))
        return oof_idx
    
    def create_cv_kfold(self,X,drugs,sig_ids,folds=folds,seed=42):
        seed_everything(seed)
        
        X = X.copy()

        X['drug_id'] = drugs
        X['sig_id'] = sig_ids
        
        mask_treatment = X.iloc[:,0].values==0
        mask_control = X.iloc[:,0].values==1
        X_with_treatment = X[mask_treatment].copy()
        X_control = X[mask_control].copy()
        
        drugs_dict = {}
        drugs_already_in_fold = []
        for i in range(folds):
            X_remaining = X_with_treatment[~X_with_treatment['drug_id'].isin(drugs_already_in_fold)].copy()

            if i<folds-1:
                tmp_drugs = self._select_split_drugs(X_remaining,1.0/(folds-i))
                drugs_already_in_fold = drugs_already_in_fold + tmp_drugs
                dd = {k:i for k in tmp_drugs}
                drugs_dict.update(dd)
            else:
                dd = {k:i for k in X_remaining['drug_id'].values.tolist()}
                drugs_dict.update(dd)
                
        
        X_with_treatment['fold'] = X_with_treatment['drug_id'].map(drugs_dict)
        X_control['fold'] = np.random.randint(0,folds,size=X_control.shape[0])
        
        X_all = pd.concat([X_with_treatment,X_control],axis=0)
        
        X_all = pd.merge(X[['sig_id']],X_all,on='sig_id')
        
        oof_assignment = X_all['fold'].values
        
        oof_idx = []
        for x in np.arange(folds):
            train = np.where(oof_assignment!=x)[0]
            val = np.where(oof_assignment==x)[0]
            oof_idx.append((train,val))
        return oof_idx
    
    
def add_control_test_to_train(prepared_data):
    X_test = prepared_data.X_test
    X_train = prepared_data.X_train
    y_train = prepared_data.y_train
    
    X_test_control = X_test[X_test['cp_type']==1]
    X_train = pd.concat([X_train,X_test_control],axis=0)

    y_test_control = pd.DataFrame(np.zeros((X_test_control.shape[0],y_train.shape[1])),columns=y_train.columns.tolist())
    y_train = pd.concat([y_train,y_test_control],axis=0)

    return X_train, y_train

In [ ]:
if nn_architecture == 'v0':
    class TabularNN(nn.Module):
        def __init__(self, cfg,last_layer_bias=None):
            super().__init__()

            self.cfg = cfg

            self.batchnorm0 = nn.BatchNorm1d(cfg.num_features)

            self.dense1 = nn.utils.weight_norm(nn.Linear(cfg.num_features, cfg.hidden_size_layer1))
            self.prelu1 = nn.PReLU()
            self.batchnorm1 = nn.BatchNorm1d(cfg.hidden_size_layer1)
            self.dropout1 = nn.Dropout(cfg.dropout)

            self.dense2 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer1, cfg.hidden_size_layer2))
            self.prelu2 = nn.PReLU()
            self.batchnorm2 = nn.BatchNorm1d(cfg.hidden_size_layer2)
            self.dropout2 = nn.Dropout(cfg.dropout)

            self.dense3 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer2, cfg.target_cols))


            if not last_layer_bias is None:
                last_layer_bias = np.log(last_layer_bias.mean(axis=0).clamp(1e-10,1))
                self.dense3.bias.data = nn.Parameter(last_layer_bias.float())

            self.params = []
            self.params += self.set_bias_weight_decay(self.batchnorm0,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.prelu1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.prelu2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense3,none_to_zero=True)


        def set_bias_weight_decay(self,layer,none_to_zero=False,all_to_zero=False):
            params = []
            named_params = dict(layer.named_parameters())
            for key, value in named_params.items():
                if none_to_zero:
                    params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
                else:
                    if key == 'bias':
                        params += [{'params':value,'weight_decay':0.0}]
                    else:
                        if all_to_zero:
                            params += [{'params':value,'weight_decay':0.0}]
                        else:
                            params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
            return params

        def recalibrate_layer(self, layer):
            if(torch.isnan(layer.weight_v).sum() > 0):
                layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
                layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

            if(torch.isnan(layer.weight).sum() > 0):
                layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
                layer.weight += 1e-7

        def forward(self, x):
            x0 = self.batchnorm0(x)

            self.recalibrate_layer(self.dense1)
            x1 = self.dense1(x0)
            x1 = self.prelu1(x1)
            x1 = self.batchnorm1(x1)
            x1 = self.dropout1(x1)

            self.recalibrate_layer(self.dense2)
            x2 = self.dense2(x1)
            x2 = self.prelu2(x2)
            x2 = self.batchnorm2(x2)
            x2 = self.dropout2(x2)

            self.recalibrate_layer(self.dense3)
            y = self.dense3(x2)
            return y

    class CFG:
        hidden_size_layer1=2048
        hidden_size_layer2=2048
        dropout=0.4
        weight_decay=1e-5
        batch_size=128
        epochs=100
        min_epochs = 25
        one_cycle_epochs=25
        number_one_cycle=1
        early_stopping=20
        learning_rate=1e-3
        patience=15
        hard_patience=25
        min_delta=0.00005
        ratio_train_val=1.15
        pct_start=0.1
        div_factor=1e3
        verbose=1
        
elif nn_architecture == 'v1':
    class TabularNN(nn.Module):
        def __init__(self, cfg,last_layer_bias=None):
            super().__init__()

            self.cfg = cfg

            self.batchnorm0 = nn.BatchNorm1d(cfg.num_features)

            self.dense1 = nn.utils.weight_norm(nn.Linear(cfg.num_features, cfg.hidden_size_layer1))
            self.prelu1 = nn.PReLU()
            self.batchnorm1 = nn.BatchNorm1d(cfg.hidden_size_layer1)
            self.dropout1 = nn.Dropout(cfg.dropout)

            self.dense2 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer1, cfg.hidden_size_layer2))
            self.prelu2 = nn.PReLU()
            self.batchnorm2 = nn.BatchNorm1d(cfg.hidden_size_layer2)
            self.dropout2 = nn.Dropout(cfg.dropout)

            self.dense3 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer2, cfg.target_cols))


            if not last_layer_bias is None:
                last_layer_bias = np.log(last_layer_bias.mean(axis=0).clamp(1e-10,1))
                self.dense3.bias.data = nn.Parameter(last_layer_bias.float())

            self.params = []
            self.params += self.set_bias_weight_decay(self.batchnorm0,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense1)
            self.params += self.set_bias_weight_decay(self.prelu1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense2)
            self.params += self.set_bias_weight_decay(self.prelu2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense3)


        def set_bias_weight_decay(self,layer,none_to_zero=False,all_to_zero=False):
            params = []
            named_params = dict(layer.named_parameters())
            for key, value in named_params.items():
                if none_to_zero:
                    params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
                else:
                    if key == 'bias':
                        params += [{'params':value,'weight_decay':0.0}]
                    else:
                        if all_to_zero:
                            params += [{'params':value,'weight_decay':0.0}]
                        else:
                            params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
            return params

        def recalibrate_layer(self, layer):
            if(torch.isnan(layer.weight_v).sum() > 0):
                layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
                layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

            if(torch.isnan(layer.weight).sum() > 0):
                layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
                layer.weight += 1e-7

        def forward(self, x):
            x0 = self.batchnorm0(x)

            self.recalibrate_layer(self.dense1)
            x1 = self.dense1(x0)
            x1 = self.prelu1(x1)
            x1 = self.batchnorm1(x1)
            x1 = self.dropout1(x1)

            self.recalibrate_layer(self.dense2)
            x2 = self.dense2(x1)
            x2 = self.prelu2(x2)
            x2 = self.batchnorm2(x2)
            x2 = self.dropout2(x2)

            self.recalibrate_layer(self.dense3)
            y = self.dense3(x2)
            return y

    class CFG:
        hidden_size_layer1=2048
        hidden_size_layer2=2048
        dropout=0.4
        weight_decay=1e-5
        batch_size=128
        epochs=100
        min_epochs = 25
        one_cycle_epochs=25
        number_one_cycle=1
        early_stopping=20
        learning_rate=1e-3
        patience=15
        hard_patience=25
        min_delta=0.00005
        ratio_train_val=1.15
        pct_start=0.1
        div_factor=1e3
        verbose=1

elif nn_architecture == 'v1.1':
    class TabularNN(nn.Module):
        def __init__(self, cfg,last_layer_bias=None):
            super().__init__()

            self.cfg = cfg

            self.batchnorm0 = nn.BatchNorm1d(cfg.num_features)

            self.dense1 = nn.utils.weight_norm(nn.Linear(cfg.num_features, cfg.hidden_size_layer1))
            self.prelu1 = nn.PReLU()
            self.batchnorm1 = nn.BatchNorm1d(cfg.hidden_size_layer1)
            self.dropout1 = nn.Dropout(cfg.dropout)

            self.dense2 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer1, cfg.hidden_size_layer2))
            self.prelu2 = nn.PReLU()
            self.batchnorm2 = nn.BatchNorm1d(cfg.hidden_size_layer2)
            self.dropout2 = nn.Dropout(cfg.dropout)

            self.dense3 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer2, cfg.target_cols))


            if not last_layer_bias is None:
                last_layer_bias = np.log(last_layer_bias.mean(axis=0).clamp(1e-10,1))
                self.dense3.bias.data = nn.Parameter(last_layer_bias.float())

            self.params = []
            self.params += self.set_bias_weight_decay(self.batchnorm0,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense1)
            self.params += self.set_bias_weight_decay(self.prelu1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense2)
            self.params += self.set_bias_weight_decay(self.prelu2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense3)


        def set_bias_weight_decay(self,layer,none_to_zero=False,all_to_zero=False):
            params = []
            named_params = dict(layer.named_parameters())
            for key, value in named_params.items():
                if none_to_zero:
                    params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
                else:
                    if key == 'bias':
                        params += [{'params':value,'weight_decay':0.0}]
                    else:
                        if all_to_zero:
                            params += [{'params':value,'weight_decay':0.0}]
                        else:
                            params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
            return params

        def recalibrate_layer(self, layer):
            if(torch.isnan(layer.weight_v).sum() > 0):
                layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
                layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

            if(torch.isnan(layer.weight).sum() > 0):
                layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
                layer.weight += 1e-7

        def forward(self, x):
            x0 = self.batchnorm0(x)

            self.recalibrate_layer(self.dense1)
            x1 = self.dense1(x0)
            x1 = self.prelu1(x1)
            x1 = self.batchnorm1(x1)
            x1 = self.dropout1(x1)

            self.recalibrate_layer(self.dense2)
            x2 = self.dense2(x1)
            x2 = self.prelu2(x2)
            x2 = self.batchnorm2(x2)
            x2 = self.dropout2(x2)

            self.recalibrate_layer(self.dense3)
            y = self.dense3(x2)
            return y

    class CFG:
        hidden_size_layer1=2048
        hidden_size_layer2=2048
        dropout=0.4
        weight_decay=1e-5
        batch_size=128
        epochs=120
        min_epochs = 25
        one_cycle_epochs=50
        number_one_cycle=1
        early_stopping=30
        learning_rate=1e-3
        patience=15
        hard_patience=25
        min_delta=0.00005
        ratio_train_val=1.15
        pct_start=0.1
        div_factor=1e3
        verbose=1
        
elif nn_architecture == 'v1.2':
    class TabularNN(nn.Module):
        def __init__(self, cfg,last_layer_bias=None):
            super().__init__()

            self.cfg = cfg

            self.batchnorm0 = nn.BatchNorm1d(cfg.num_features)

            self.dense1 = nn.utils.weight_norm(nn.Linear(cfg.num_features, cfg.hidden_size_layer1))
            self.prelu1 = nn.PReLU()
            self.batchnorm1 = nn.BatchNorm1d(cfg.hidden_size_layer1)
            self.dropout1 = nn.Dropout(cfg.dropout)

            self.dense2 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer1, cfg.hidden_size_layer2))
            self.prelu2 = nn.PReLU()
            self.batchnorm2 = nn.BatchNorm1d(cfg.hidden_size_layer2)
            self.dropout2 = nn.Dropout(cfg.dropout)

            self.dense3 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer2, cfg.target_cols))


            if not last_layer_bias is None:
                last_layer_bias = np.log(last_layer_bias.mean(axis=0).clamp(1e-10,1))
                self.dense3.bias.data = nn.Parameter(last_layer_bias.float())

            self.params = []
            self.params += self.set_bias_weight_decay(self.batchnorm0,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense1)
            self.params += self.set_bias_weight_decay(self.prelu1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense2)
            self.params += self.set_bias_weight_decay(self.prelu2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense3)


        def set_bias_weight_decay(self,layer,none_to_zero=False,all_to_zero=False):
            params = []
            named_params = dict(layer.named_parameters())
            for key, value in named_params.items():
                if none_to_zero:
                    params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
                else:
                    if key == 'bias':
                        params += [{'params':value,'weight_decay':0.0}]
                    else:
                        if all_to_zero:
                            params += [{'params':value,'weight_decay':0.0}]
                        else:
                            params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
            return params

        def recalibrate_layer(self, layer):
            if(torch.isnan(layer.weight_v).sum() > 0):
                layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
                layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

            if(torch.isnan(layer.weight).sum() > 0):
                layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
                layer.weight += 1e-7

        def forward(self, x):
            x0 = self.batchnorm0(x)

            self.recalibrate_layer(self.dense1)
            x1 = self.dense1(x0)
            x1 = self.prelu1(x1)
            x1 = self.batchnorm1(x1)
            x1 = self.dropout1(x1)

            self.recalibrate_layer(self.dense2)
            x2 = self.dense2(x1)
            x2 = self.prelu2(x2)
            x2 = self.batchnorm2(x2)
            x2 = self.dropout2(x2)

            self.recalibrate_layer(self.dense3)
            y = self.dense3(x2)
            return y

    class CFG:
        hidden_size_layer1=2048
        hidden_size_layer2=2048
        dropout=0.4
        weight_decay=1e-5
        batch_size=128
        epochs=120
        min_epochs = 40
        one_cycle_epochs=100
        number_one_cycle=1
        early_stopping=30
        learning_rate=1e-3
        patience=30
        hard_patience=25
        min_delta=0.00005
        ratio_train_val=1.15
        pct_start=0.1
        div_factor=1e3
        verbose=1

elif nn_architecture == 'v2':
    class TabularNN(nn.Module):
        def __init__(self, cfg,last_layer_bias=None):
            super().__init__()

            self.cfg = cfg

            self.batchnorm0 = nn.BatchNorm1d(cfg.num_features)

            self.dense1 = nn.utils.weight_norm(nn.Linear(cfg.num_features, cfg.hidden_size_layer1))
            self.prelu1 = nn.PReLU()
            self.batchnorm1 = nn.BatchNorm1d(cfg.hidden_size_layer1)
            self.dropout1 = nn.Dropout(cfg.dropout)

            self.dense2 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer1, cfg.hidden_size_layer2))
            self.prelu2 = nn.PReLU()
            self.batchnorm2 = nn.BatchNorm1d(cfg.hidden_size_layer2)
            self.dropout2 = nn.Dropout(cfg.dropout)

            self.dense3 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size_layer2, cfg.target_cols))


            if not last_layer_bias is None:
                last_layer_bias = np.log(last_layer_bias.mean(axis=0).clamp(1e-10,1))
                self.dense3.bias.data = nn.Parameter(last_layer_bias.float())

            self.params = []
            self.params += self.set_bias_weight_decay(self.batchnorm0,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense1)
            self.params += self.set_bias_weight_decay(self.prelu1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm1,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout1,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense2)
            self.params += self.set_bias_weight_decay(self.prelu2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.batchnorm2,none_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dropout2,all_to_zero=True)
            self.params += self.set_bias_weight_decay(self.dense3)


        def set_bias_weight_decay(self,layer,none_to_zero=False,all_to_zero=False):
            params = []
            named_params = dict(layer.named_parameters())
            for key, value in named_params.items():
                if none_to_zero:
                    params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
                else:
                    if key == 'bias':
                        params += [{'params':value,'weight_decay':0.0}]
                    else:
                        if all_to_zero:
                            params += [{'params':value,'weight_decay':0.0}]
                        else:
                            params += [{'params':value,'weight_decay':self.cfg.weight_decay}]
            return params

        def recalibrate_layer(self, layer):
            if(torch.isnan(layer.weight_v).sum() > 0):
                layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
                layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

            if(torch.isnan(layer.weight).sum() > 0):
                layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
                layer.weight += 1e-7

        def forward(self, x):
            x0 = self.batchnorm0(x)

            self.recalibrate_layer(self.dense1)
            x1 = self.dense1(x0)
            x1 = self.prelu1(x1)
            x1 = self.batchnorm1(x1)
            x1 = self.dropout1(x1)

            self.recalibrate_layer(self.dense2)
            x2 = self.dense2(x1)
            x2 = self.prelu2(x2)
            x2 = self.batchnorm2(x2)
            x2 = self.dropout2(x2)

            self.recalibrate_layer(self.dense3)
            y = self.dense3(x2)
            return y

    class CFG:
        hidden_size_layer1=1024
        hidden_size_layer2=1024
        dropout=0.4
        weight_decay=1e-5
        batch_size=128
        epochs=100
        min_epochs = 25
        one_cycle_epochs=25
        number_one_cycle=1
        early_stopping=20
        learning_rate=1e-3
        patience=15
        hard_patience=25
        min_delta=0.00005
        ratio_train_val=1.15
        pct_start=0.1
        div_factor=1e3
        verbose=1
    
    
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing
    def forward(self, x, target):
        confidence = 1. - self.smoothing
        logprobs = F.log_softmax(x, dim=-1)
        bcs_loss = nn.BCEWithLogitsLoss()(x, target)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * bcs_loss + self.smoothing * smooth_loss
        return loss.mean()
    
    
class NNWrapper():

    def __init__(self,model_class,cfg):
        self.model_class = model_class
        self.cfg = cfg
        self.clamp = 1e-7 #To avoid having 0 or 1 in logloss
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        
    def _evaluate(self,y_actual,y_pred):
        if set([0,1])==set(self.labels):
            return metrics.log_loss(y_actual.reshape(-1,1),y_pred.reshape(-1,1),labels=self.labels)
        else:
            return -np.sum(y_actual.reshape(-1,1)*np.log(y_pred.reshape(-1,1)) + (1-y_actual.reshape(-1,1))*np.log(1-y_pred.reshape(-1,1)))/y_pred.reshape(-1,1).shape[0]
        
    def _numpy_to_tensor(self,n):
        t = (torch.from_numpy(n)).float()
        return t
    
    def _index_to_boolean(self,idx,size):
            mask_array = np.zeros(size)
            mask_array = mask_array==1
            mask_array[idx] = True
            return mask_array
        
    def fit(self,X,y,X_holdout=None,y_holdout=None,folds=5,params=None,evaluate=True,oof_idx=None,seed=42,moa_control_params=None):
        seed_everything(seed=seed)
        #Information on data
        self.labels = list(np.unique(y))
        
        
        #Parameters particular to MoA
        self.moa_control_params = moa_control_params
        
        
        if oof_idx is None:
            self._oof_idx = []
            if (y.ndim == 2) and (y.shape[1] > 1):
                cv = KFold(n_splits=folds, shuffle=True, random_state=SEED)
                if not self.moa_control_params is None:
                    if ['fold_type']=='multi':
                        cv = MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)
            else:
                cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)
            for (train_idx, val_idx) in cv.split(X,y):
                self._oof_idx.append((train_idx,val_idx))
        else:
            self._oof_idx = oof_idx
        
        #Train models
        self.cv_models = []
        self._X = X.copy()
        self.total_oof_loss = 0
        self._y = y
        for fold, (train_idx, val_idx) in enumerate(self._oof_idx):
            print("Start training fold {} of {}".format(fold+1,folds))
            if self.moa_control_params is None:
                X_train, X_val = X[train_idx], X[val_idx]
                y_train, y_val = y[train_idx], y[val_idx]
            else:
                train_mask = self._index_to_boolean(train_idx,X.shape[0])
                val_mask = self._index_to_boolean(val_idx,X.shape[0])
                
                train_mask_non_control = train_mask & self.moa_control_params['mask_treatment']
                self.train_mask_non_control = train_mask_non_control
                train_mask_control = train_mask & (self.moa_control_params['mask_treatment']==False)
                val_mask_non_control = val_mask & self.moa_control_params['mask_treatment']
                
                X_train, X_val = X[train_mask_non_control], X[val_mask_non_control]
                y_train, y_val = y[train_mask_non_control], y[val_mask_non_control]
                
                if self.moa_control_params['add_control_from_test']:
                    self.X_control = np.concatenate([X[train_mask_control],self.moa_control_params['test_control_data']],axis=0)
                else:
                    self.X_control = X[train_mask_control]
            
            model = self._train(X_train,y_train,X_val,y_val)
            self.cv_models.append(model)
            
            if evaluate:
                #Evaluate fold model
                oof_preds = self._predict_proba_model(model,X_val)
                oof_loss = self._evaluate(y_val,oof_preds)
                self.total_oof_loss = self.total_oof_loss + oof_loss/folds
                if not X_holdout is None:
                    holdout_preds = self._predict_proba_model(model,X_holdout)
                    holdout_loss = self._evaluate(y_holdout,holdout_preds)

                if not X_holdout is None:
                    print('Fold {} out of fold score: {:.6f}; holdout score: {:.6f}'.format(fold+1,oof_loss,holdout_loss))
                else:
                    print('Fold {} out of fold score: {:.6f}'.format(fold+1,oof_loss))
             
        #Evaluate whole model
        if evaluate:
            if not X_holdout is None:
                y_pred_holdout = self.predict(X_holdout)
                holdout_loss = self._evaluate(params,y_holdout,y_pred_holdout)
            if not X_holdout is None:
                print('Total oof score: {:.6f}; holdout score: {:.6f}'.format(self.total_oof_loss,holdout_loss))
            else:
                print('Total oof score: {:.6f}'.format(self.total_oof_loss))

            
    def _prepare_batch_data(self,X,y=None,inference=False):
        if inference:
            dataset = DataLoader(TensorDataset(X),batch_size=X.shape[0],shuffle=False,pin_memory=True,num_workers=multiprocessing.cpu_count()-1)
        else:
            #Before passing it to DataLoader, X,y need to be in the following dimension: items, features
            dataset = DataLoader(TensorDataset(X,y),batch_size=self.cfg.batch_size,shuffle=True,pin_memory=True,num_workers=multiprocessing.cpu_count()-1,drop_last=True)
        return dataset
    
    
    def _calculate_controle_size_to_add(self,X_train):
        #Calculate how many we take from control pool
        control_size_to_add = min(math.floor(X_train.shape[0]*self.moa_control_params['control_share']),self.X_control.shape[0])
        print('Add {} control to training set, representing {:.0f}% of vehicle size'.format(control_size_to_add,100*control_size_to_add/X_train.shape[0]))
        return control_size_to_add
    
    def _moa_add_control_data(self,X_train,y_train,control_size_to_add):
        
        if control_size_to_add>0:
            mask_control_to_add = np.random.choice(np.arange(0,self.X_control.shape[0]), size=control_size_to_add, replace=False)
            X_control_to_add = self.X_control[mask_control_to_add]
            X_train = np.concatenate([X_train,X_control_to_add],axis=0)

            y_control_to_add = np.zeros((X_control_to_add.shape[0],y_train.shape[1]),dtype=y_train.dtype)
            y_train = np.concatenate([y_train,y_control_to_add],axis=0)
        
        X_train_torch = self._numpy_to_tensor(X_train)
        y_train_torch = self._numpy_to_tensor(y_train)
        
        return X_train_torch,y_train_torch
    
    
    def _augment_data(self,X,var_list,mask,epoch):
        var_size = len(var_list)
        i = (epoch-1)%var_size
        var = var_list[i]
        var_val = var[mask]
        X = X + var_val
        return X
        
            
    def _train(self, X_train, y_train, X_val, y_val):
        initial_time = datetime.datetime.now()
        
        X_train_size = X_train.shape[0]

        if not self.moa_control_params is None:
            control_size_to_add = self._calculate_controle_size_to_add(X_train)
            X_train_size += control_size_to_add
        
        #Format data
        X_train_torch = self._numpy_to_tensor(X_train)
        y_train_torch = self._numpy_to_tensor(y_train)
        X_val_torch = self._numpy_to_tensor(X_val)
        y_val_torch = self._numpy_to_tensor(y_val)
        
        #Initialize model
        model = self.model_class(self.cfg,y_train_torch)
        model.to(self.device)
        
        #Initialize model parameters
        loss_fn = torch.nn.BCEWithLogitsLoss(reduction = 'mean')
        loss_val_fn = torch.nn.BCEWithLogitsLoss(reduction = 'mean')
        
        if not self.moa_control_params is None:
            if self.moa_control_params['label_smoothing']>0:
                loss_fn = LabelSmoothingCrossEntropy(self.moa_control_params['label_smoothing'])
        
        optimizer = torch.optim.Adam(model.params, lr=self.cfg.learning_rate, weight_decay=self.cfg.weight_decay)
        
        
        total_steps_oneCycle = (int(X_train_size/self.cfg.batch_size)+1)*self.cfg.one_cycle_epochs
        continue_OneCycleLR = True
        scheduler2 = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.3, div_factor=2, 
                                              max_lr=self.cfg.learning_rate*5, total_steps=total_steps_oneCycle,
                                              )
        scheduler1 = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=self.cfg.patience, factor=0.9, threshold=1e-5)
    
        #Print parameters
        if hasattr(self.cfg, 'verbose'):
            verbose = self.cfg.verbose
        else:
            verbose = 1
        
        #Train model
        
        for epoch in range(self.cfg.epochs):
            model.train()
            avg_loss = 0.0
            
            if not self.moa_control_params is None:
                if (self.moa_control_params['augment_data']) & (epoch>0):
                    X_train_tmp = self._augment_data(X_train,
                                                     self.moa_control_params['augment_var_list'],
                                                     self.train_mask_non_control,
                                                     epoch
                                                    )
                else:
                    X_train_tmp = X_train.copy()
                X_train_torch, y_train_torch = self._moa_add_control_data(X_train_tmp, y_train, control_size_to_add)
            
            #We create the train batch dataset here to be able to shuffle it
            dataset = self._prepare_batch_data(X_train_torch,y_train_torch)
            
            #Train trhough batches
            for X_batch, y_batch in dataset:
                X_batch = X_batch.to(self.device)
                y_batch = y_batch.to(self.device)
                # Forward pass: compute predicted y by passing x to the model.
                y_pred = model(X_batch)
                
                loss = loss_fn(y_pred, y_batch)
                avg_loss += loss.item() / (len(dataset))
                
                # Before the backward pass, use the optimizer object to zero all of the
                # gradients for the variables it will update (which are the learnable
                # weights of the model). This is because by default, gradients are
                # accumulated in buffers( i.e, not overwritten) whenever .backward()
                # is called. Checkout docs of torch.autograd.backward for more details.
                optimizer.zero_grad()
                
                # Backward pass: compute gradient of the loss with respect to model
                # parameters
                loss.backward()
                
                # Calling the step function on an Optimizer makes an update to its
                # parameters
                optimizer.step()
                if (continue_OneCycleLR) and (self.cfg.number_one_cycle>0):
                    scheduler2.step()

            #OneCycle update
            if (self.cfg.number_one_cycle==0):
                continue_OneCycleLR = False
            if (continue_OneCycleLR):
                if (epoch+1)%(self.cfg.one_cycle_epochs*self.cfg.number_one_cycle)==0:
                    print('Finish One Cycle')
                    continue_OneCycleLR=False
                    for g in optimizer.param_groups:
                        g['lr'] = self.cfg.learning_rate
                    
                if ((epoch+1)%self.cfg.one_cycle_epochs==0) and (epoch+1)<(self.cfg.one_cycle_epochs*self.cfg.number_one_cycle):
                    print('Reinitialize One Cycle')
                    scheduler2 = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.3, div_factor=2, 
                                              max_lr=self.cfg.learning_rate*5, total_steps=total_steps_oneCycle,
                                              )
            
            
            #Evaluation
            model.eval()
            y_pred_val = self._predict_proba_model(model,X_val_torch,for_loss=True)
            loss_val = loss_val_fn(y_pred_val, y_val_torch).item()
            
            check_early_stopping, is_new_best = self._should_stop(epoch,loss_val,model,self.cfg.early_stopping,self.cfg.min_delta,avg_loss,self.cfg.min_epochs,self.cfg.hard_patience,self.cfg.ratio_train_val)
            if is_new_best:
                text_new_best = 'New Best'
            else:
                text_new_best = ''
            if (epoch+1)%verbose == 0:
                print('Epoch {}/{} \t train loss: {:.5f} \t val loss: {:.5f} \t time: {}s \t lr: {:.6f} \t {}'.format(epoch+1,self.cfg.epochs,avg_loss,loss_val, (datetime.datetime.now() - initial_time).seconds, optimizer.param_groups[-1]['lr'],text_new_best))
                pass
                
            if not self.cfg.early_stopping is None:
                if check_early_stopping:
                    print("Early stopping, best epoch: {}".format(self._best_epoch+1))
                    model_to_return = self._load_best_model()
                    return model_to_return
                
            if not self.moa_control_params is None: 
                if loss_val < self.moa_control_params['oof_loss_limit']:
                    print('Model decrease below limit')
                    return model

            #Update lr when oneCycle is over
            if not continue_OneCycleLR:
                scheduler1.step(loss_val)
        
        model_to_return = self._load_best_model()
        
        return model_to_return
    
    def _load_best_model(self):
        model_to_return = self.model_class(self.cfg)
        model_to_return.to(self.device)
        model_to_return.load_state_dict(torch.load('tmp_model_state_dict'))
        return model_to_return
    
    
    def save_model(self,name=''):
        for i,model in enumerate(self.cv_models):
            torch.save(model.state_dict(), name+'cv_'+str(i))
        
    def load_model(self,folds,name=''):
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.cv_models = []
        for i in range(folds):
            model = self.model_class(self.cfg)
            model.to(self.device)
            model.load_state_dict(torch.load(name+'cv_'+str(i)))
            self.cv_models.append(model)
            
    def load_oof_idx(self,oof_idx):
        self._oof_idx = oof_idx
    def load_X(self,X):
        self._X = X
            
    def _should_stop(self,epoch,val_loss,model,patience,min_delta,train_loss,min_epochs=None,hard_patience=None,ratio_train_val=1.1):
        if min_epochs is None:
            min_epochs = 0
        if hard_patience is None:
            hard_patience = epoch + 1
        if epoch==0:
            self._best_val_loss = val_loss
            #self._best_model = copy.deepcopy(model)
            torch.save(model.state_dict(), 'tmp_model_state_dict')
            self._best_epoch = epoch
            self.epochs_since_best = 1
        if val_loss < self._best_val_loss - min_delta:
            self._best_val_loss = val_loss
            #self._best_model = copy.deepcopy(model)
            torch.save(model.state_dict(), 'tmp_model_state_dict')
            self._best_epoch = epoch
            self.epochs_since_best = 1
            return False, True
        else:
            self.epochs_since_best += 1
            if (epoch - self._best_epoch > patience) and (epoch > min_epochs) and ((val_loss>train_loss*ratio_train_val) or (epoch - self._best_epoch > hard_patience)):
                return True, False
            else:
                return False, False

    def _predict_proba_model(self, model, X, for_loss=False):
        if isinstance(X,np.ndarray):
            X = self._numpy_to_tensor(X)
            
        y_pred = []
        
        dataset = self._prepare_batch_data(X,inference=True)
        
        model.eval()
        with torch.no_grad():
            for X_batch in dataset:
                y = model(X_batch[0].to(self.device))
                if for_loss:
                    y_pred.append(y.detach().cpu())
                else:
                    if self.clamp is None:
                        y_pred.append(y.sigmoid().detach().cpu().numpy())
                    else:
                        y_pred.append(y.sigmoid().clamp(self.clamp,1-self.clamp).detach().cpu().numpy())  
        if for_loss:
            y_pred = torch.cat(y_pred,dim=0)
        else:
            y_pred = np.concatenate(y_pred)
        return y_pred
        
    def predict(self,X):
        output_dim = self.cfg.target_cols
        
        y = np.zeros((X.shape[0],output_dim))
        y = np.repeat(y[..., np.newaxis], len(self.cv_models), axis=-1)
        for i,model in enumerate(self.cv_models):
            y[...,i] = self._predict_proba_model(model,X)
        y = y.mean(axis=-1)
        return y
    
    def predict_oof(self):
        y = (self._y.copy()).astype(float)
        for (train_idx, val_idx), model in zip(self._oof_idx, self.cv_models):
            y[val_idx,...] = self._predict_proba_model(model,self._X[val_idx])
        return y

In [ ]:
def add_nonscored_labels_pred(X_train,nn_model_non_scored,oof_idx):
    X_train_nonscored_pred = np.append(X_train,np.zeros((X_train.shape[0],402)),axis=1)
    for fold, (train_idx, val_idx) in enumerate(oof_idx):
        X = X_train[val_idx,:]
        nonscored_pred = nn_model_non_scored._predict_proba_model(nn_model_non_scored.cv_models[fold],X)
        X_train_nonscored_pred[val_idx,-402:] = nonscored_pred
    return X_train_nonscored_pred

In [ ]:
def mask_variation(var_list,mask):
    var_list_new = []
    for var in var_list:
        var_list_new.append(var[mask].copy())
    return var_list_new

# Run

In [ ]:
#%debug
if (run_script) and (calculate_new_prepared_data):
    prepared_data = prepareData(path_data,validation_ratio=validation_ratio,folds=folds, cpu=cpu, normalization_type=normalization_type,
                                g_removal_count=g_removal_count,exclude_c_from_kde=exclude_c_from_kde,exclude_g_from_kde=exclude_g_from_kde,add_c_stats=add_c_stats,
                                add_kernels=add_kernels, use_diff_kde=use_diff_kde, use_train_test_for_norm=use_train_test_for_norm,
                                perform_pca=perform_pca, pca_variance_threshold=0.95,pca_for_kde=pca_for_kde,pca_for_c=pca_for_c,
                                use_log_for_kernel_diff=use_log_for_kernel_diff, inverse_kde=inverse_kde, ratio_inverse_kde=ratio_inverse_kde,
                                granularity=granularity,max_dev=max_dev,normal_std_dev=normal_std_dev,additional=additional
                                )

In [ ]:
if (run_script) and (calculate_new_prepared_data):
    if dump_prepared_data:
        pickle_dump(prepared_data,prepared_data_version)
    if is_submission:
        prepared_data.prepare_test_data(path_data)

In [ ]:
if (run_script) and (calculate_new_prepared_data==False):
    if is_kaggle:
        prepared_data = pickle_load('../input/moa-'+prepared_data_version+'/'+prepared_data_version)
    else:
        prepared_data = pickle_load(prepared_data_version)
    if is_submission:
        prepared_data.prepare_test_data(path_data)

In [ ]:
#%debug

#Needs to be set after feature creation to avoid pickling issues
torch.backends.cudnn.deterministic = True
if run_script and train_model:
    nn_model_list = []
    for i in range(n_seeds):
        seed = SEED + i
        print('\n')
        print('START WITH SEED: {}'.format(seed))

        X_train,y_train,X_drugs,X_sig_id,X_holdout,y_holdout,mask_train = prepared_data.split_train_holdout(validation_ratio=validation_ratio,seed=seed,threshold=oof_threshold)
        if oof_type=='multi':
            oof_idx = prepared_data.create_cv(X_train,y_train,X_drugs,X_sig_id,folds=folds,seed=seed,threshold=oof_threshold)
        else:
            oof_idx = prepared_data.create_cv_kfold(X_train,X_drugs,X_sig_id,folds=folds,seed=seed)

        X_test_control = prepared_data.X_test.values
        mask = X_test_control[:,1]==1
        X_test_control = X_test_control[mask][:,4:].copy()
        X_test_control = X_test_control.astype(np.float32)

        X = X_train.values
        y = y_train.values

        #Get treatment
        mask_treatment = X[:,0]==0

        #Keep only useful columns
        X = X[:,3:].copy()

        CFG.num_features=X.shape[1]
        CFG.target_cols=y.shape[1]
        
        
        #Data augmentation
        var_list = None
        if augment_data:
            del var_list
            gc.collect()
            var_list = mask_variation(prepared_data.var_list,mask_train)


        moa_control_params = {
            'control_share': control_share_in_train,
            'add_control_from_test': add_control_from_test,
            'mask_treatment': mask_treatment,
            'test_control_data': X_test_control,
            'augment_data': augment_data,
            'augment_var_list': var_list,
            'label_smoothing': label_smoothing,
            'oof_loss_limit': oof_loss_limit,
            'fold_type':fold_type
        }

        nn_model = NNWrapper(TabularNN,CFG)
        nn_model.fit(X,y,folds=folds,evaluate=False,oof_idx=oof_idx,moa_control_params=moa_control_params,seed=seed)

        dict_model = {
            'nn_model':nn_model,
            'X_train':X_train,
            'y_train':y_train,
            'X_holdout':X_holdout,
            'y_holdout':y_holdout
        }
        nn_model_list.append(dict_model)

In [ ]:
if train_model:
    for i,model_dict in enumerate(nn_model_list):
        model_dict['nn_model'].save_model(name='seed_'+str(i)+'_')

In [ ]:
class post_process():
    def __init__(self,X,y):
        self.X = X.copy()
        self.y = y.copy()
        
    def control_to_zero(self,y=None):
        if y is None:
            y = self.y.copy()
        control_idx = self.X[:,0]==1
        y[control_idx,:] = 0
        return y
    
    def label_smoothing(self,ls_floor,ls_ceil,with_control=True):
        y = self.y.copy()
        y = y.clip(ls_floor,1-ls_ceil)
        if with_control:
            y = self.control_to_zero(y)
        return y
    
    def exclusivity(self,exclusivity_tuples_idx,ls_floor,ls_ceil,activation_threshold=0.3,floor=1e-5):
        y = self.y.copy()
        y = self.label_smoothing(ls_floor,ls_ceil)
        
        for tup in exclusivity_tuples_idx:
            y_tmp = y[:,tup]

            max_val = np.amax(y_tmp,axis=1)
            max_mask = max_val>=activation_threshold
            max_mask = np.repeat(np.array([max_mask]),y_tmp.shape[1],axis=0).transpose()

            max_idx = np.argmax(y_tmp,axis=1)
            max_idx = np.repeat(np.array([max_idx]),y_tmp.shape[1],axis=0).transpose()
            y_idx = np.repeat(np.array([np.arange(0,len(tup))]),y_tmp.shape[0],axis=0)

            y_mask = (y_idx!=max_idx) & (max_mask)
            y_global_mask = y==-1
            y_global_mask[:,tup] = y_mask
            y[y_global_mask] = np.clip(y[y_global_mask],0,floor)
        return y
    
def get_available_exclusivity_tuples(prepared_data,
                                     model_nonscored=model_nonscored,
                                     exclusivity_tuples=exclusivity_tuples
                                     ):
    available_cols = prepared_data.y_train.columns.tolist()
    if model_nonscored:
        available_cols = available_cols + prepared_data.X_train_nonscored.columns.tolist()

    exclusivity_tuples_idx = []
    for tup in exclusivity_tuples:
        if all([x in available_cols for x in tup]):
            idx = []
            for col in tup:
                idx.append(available_cols.index(col))
            exclusivity_tuples_idx.append(idx)

    return exclusivity_tuples_idx

In [ ]:
def evaluate_nn_model_list(nn_model_list,ls_floor=1e-5,ls_ceil=1e-5,activation_threshold=0.3,floor=1e-5):
    results = []
    for model_dict in nn_model_list:
        results.append(
            evaluate_nn_model(
                model_dict['X_train'],
                model_dict['y_train'],
                model_dict['X_holdout'],
                model_dict['y_holdout'],
                model_dict['nn_model'],
                ls_floor,
                ls_ceil,
                activation_threshold,
                floor
            )
        )
    results = np.array(results)
    results_mean = results.mean(axis=0)
    print("oof: {:.6f} \t oof post: {:.6f} \t oof ls: {:.6f} \t oof excl: {:.6f} \t holdout: {:.6f} \t holdout post: {:.6f} \t holdout ls: {:.6f} \t holdout excl: {:.6f}".format(
        results_mean[0],results_mean[1],results_mean[2],results_mean[3],
        results_mean[4],results_mean[5],results_mean[6],results_mean[7]
    ))

def evaluate_nn_model(X_train,y_train,X_holdout,y_holdout,nn_model,ls_floor=1e-5,ls_ceil=1e-5,activation_threshold=0.3,floor=1e-5):
    exclusivity_tuples_idx = get_available_exclusivity_tuples(prepared_data)
    #OOF
    y_oof_pred = (y_train.values.copy()).astype(float)
    X_oof = X_train.values 
    y_oof_pred_all = y_oof_pred
    X_oof_all = X_oof
    
    for (train_idx, val_idx), model in zip(nn_model._oof_idx, nn_model.cv_models):
        y_oof_pred[val_idx,...] = nn_model._predict_proba_model(model,nn_model._X[val_idx])
    
    postProcess_oof = post_process(X_oof_all,y_oof_pred_all)
    y_oof_post_processed = postProcess_oof.control_to_zero()
    y_oof_post_processed_ls = postProcess_oof.label_smoothing(ls_floor,ls_ceil)
    y_oof_post_processed_exclusivity = postProcess_oof.exclusivity(exclusivity_tuples_idx,ls_floor,ls_ceil,activation_threshold,floor)
    
    oof_loss = metrics.log_loss(y_train.values.reshape(-1,1),y_oof_pred_all.reshape(-1,1))
    oof_loss_post_processed = metrics.log_loss(y_train.values.reshape(-1,1),y_oof_post_processed.reshape(-1,1))
    oof_loss_post_processed_ls = metrics.log_loss(y_train.values.reshape(-1,1),y_oof_post_processed_ls.reshape(-1,1))
    oof_loss_post_processed_exclusivity = metrics.log_loss(y_train.values.reshape(-1,1),y_oof_post_processed_exclusivity.reshape(-1,1))
    
    #Holdout
    y_holdout_pred = nn_model.predict(X_holdout.values[:,3:])
    
    postProcess_holdout = post_process(X_holdout.values,y_holdout_pred)
    
    y_holdout_pred_post_processed = postProcess_holdout.control_to_zero()
    y_holdout_pred_post_processed_ls = postProcess_holdout.label_smoothing(ls_floor,ls_ceil)
    y_holdout_pred_post_processed_exclusivity = postProcess_holdout.exclusivity(exclusivity_tuples_idx,ls_floor,ls_ceil,activation_threshold,floor)
    
    holdout_loss = metrics.log_loss(y_holdout.values.reshape(-1,1),y_holdout_pred.reshape(-1,1))
    holdout_loss_post_processed = metrics.log_loss(y_holdout.values.reshape(-1,1),y_holdout_pred_post_processed.reshape(-1,1))
    holdout_loss_post_processed_ls = metrics.log_loss(y_holdout.values.reshape(-1,1),y_holdout_pred_post_processed_ls.reshape(-1,1))
    holdout_loss_post_processed_exclusivity = metrics.log_loss(y_holdout.values.reshape(-1,1),y_holdout_pred_post_processed_exclusivity.reshape(-1,1))
    
#     print("oof: {:.6f} \t oof post: {:.6f} \t oof ls: {:.6f} \t oof excl: {:.6f} \t holdout: {:.6f} \t holdout post: {:.6f} \t holdout ls: {:.6f} \t holdout excl: {:.6f}".format(
#         oof_loss,oof_loss_post_processed,oof_loss_post_processed_ls,oof_loss_post_processed_exclusivity,
#         holdout_loss,holdout_loss_post_processed,holdout_loss_post_processed_ls,holdout_loss_post_processed_exclusivity
#     ))
    return [oof_loss,oof_loss_post_processed,oof_loss_post_processed_ls,oof_loss_post_processed_exclusivity,
        holdout_loss,holdout_loss_post_processed,holdout_loss_post_processed_ls,holdout_loss_post_processed_exclusivity]

In [ ]:
def make_submission(prepared_data,nn_model_list,ls_floor=1e-5,ls_ceil=1e-5,cfg=CFG,path_data=path_data):
    preds_list = []
    for model_dict in nn_model_list:
        preds_list.append(
            model_dict['nn_model'].predict(prepared_data.X_test.values[:,4:].astype(np.float32))
        )
    preds = np.array(preds_list)
    preds = preds.mean(axis=0)
    
    postProcess = post_process(prepared_data.X_test.iloc[:,1:].values,preds)
    preds = postProcess.label_smoothing(ls_floor,ls_ceil)
    submission = pd.read_csv(path_data+'sample_submission.csv')
    submission.iloc[:,1:] = preds
    submission.to_csv('submission.csv',index=False)
    return submission

In [ ]:
if train_model==False:
    nn_model_list = []
    for i in range(n_seeds):
        seed = 42 + i
        X_train,y_train,X_drugs,X_sig_id,X_holdout,y_holdout,mask_train = prepared_data.split_train_holdout(validation_ratio=validation_ratio,seed=seed,threshold=oof_threshold)
        if oof_type=='multi':
            oof_idx = prepared_data.create_cv(X_train,y_train,X_drugs,X_sig_id,folds=folds,seed=seed,threshold=oof_threshold)
        else:
            oof_idx = prepared_data.create_cv_kfold(X_train,X_drugs,X_sig_id,folds=folds,seed=seed)
        
        CFG.num_features=X_train.values[:,3:].shape[1]
        CFG.target_cols=y_train.shape[1]
        
        nn_model = NNWrapper(TabularNN,CFG)
        nn_model.load_model(folds,path_model+'seed_'+str(i)+'_')
        nn_model.load_oof_idx(oof_idx)
        nn_model.load_X(X_train.values[:,3:])
        
        dict_model = {
            'nn_model':nn_model,
            'X_train':X_train,
            'y_train':y_train,
            'X_holdout':X_holdout,
            'y_holdout':y_holdout
        }
        
        nn_model_list.append(dict_model)

In [ ]:
if run_script:
    if (is_submission):
        make_submission(prepared_data,nn_model_list)
    if (not is_submission) and (not is_train_without_holdout):
        evaluate_nn_model_list(nn_model_list,ls_floor=1e-5,ls_ceil=1e-5,activation_threshold=0.3,floor=1e-7)